In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
from bs4 import BeautifulSoup
import requests
def download_ggm(model_name: str='GO_CONS_GCF_2_TIM_R6e'):
    '''
    Download static gravity model from ICGEM
    
    Parameters
    ----------
    model_name: (str) Name of global model
    
    Returns
    -------
    None
    
    Notes
    -----
    1. Automatically writes global model to file
    '''
    base_url = "https://icgem.gfz-potsdam.de/tom_longtime"
    model_url = 'https://icgem.gfz-potsdam.de'
    response = requests.get(base_url)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        # Find the URL that corresponds to the model_name
        for link in soup.find_all('a'):
            if model_name in link.get('href') and 'gfc' in link.get('href'):
                model_url += link.get('href')
                response = requests.get(model_url)
                with open(model_name+'.gfc', 'w') as f:
                    f.writelines(response.text)
    return

In [5]:
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm

def download_ggm(model_name: str='GO_CONS_GCF_2_TIM_R6e'):
    '''
    Download static gravity model from ICGEM
    
    Parameters
    ----------
    model_name: (str) Name of global model
    
    Returns
    -------
    None
    
    Notes
    -----
    1. Automatically writes global model to file
    '''
    base_url = "https://icgem.gfz-potsdam.de/tom_longtime"
    model_url = 'https://icgem.gfz-potsdam.de'
    response = requests.get(base_url)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        # Find the URL that corresponds to the model_name
        for link in soup.find_all('a'):
            if model_name in link.get('href') and 'gfc' in link.get('href'):
                model_url += link.get('href')
                response = requests.get(model_url, stream=True)
                total_size = int(response.headers.get('content-length', 0))
                block_size = 1024 #1 Kibibyte
                t=tqdm(total=total_size, unit='iB', unit_scale=True)
                with open(model_name+'.gfc', 'wb') as f:
                    for data in response.iter_content(block_size):
                        t.update(len(data))
                        f.write(data)
                t.close()
                if total_size != 0 and t.n != total_size:
                    print("ERROR, something went wrong")
    return


In [8]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import os

def download_ggm(model_name: str = 'GO_CONS_GCF_2_TIM_R6e'):
    '''
    Download static gravity model from ICGEM
    
    Parameters
    ----------
    model_name: (str) Name of global model
    
    Returns
    -------
    None
    
    Notes
    -----
    1. Automatically writes global model to file
    '''
    base_url = "https://icgem.gfz-potsdam.de/tom_longtime"
    model_url_prefix = 'https://icgem.gfz-potsdam.de'

    try:
        response = requests.get(base_url)
        response.raise_for_status()  # Raise an HTTPError for bad responses (4xx and 5xx)
    except requests.RequestException as e:
        print(f"Error fetching base URL: {e}")
        return

    soup = BeautifulSoup(response.text, 'html.parser')
    
    model_url = None
    for link in soup.find_all('a', href=True):
        if model_name in link['href'] and 'gfc' in link['href']:
            model_url = model_url_prefix + link['href']
            break

    if not model_url:
        print(f"Model {model_name} not found.")
        return

    try:
        response = requests.get(model_url, stream=True)
        response.raise_for_status()
    except requests.RequestException as e:
        print(f"Error fetching model URL: {e}")
        return

    total_size = int(response.headers.get('content-length', 0))
    block_size = 1024  # 1 Kibibyte

    # Ensure output directory exists
    os.makedirs('downloads', exist_ok=True)
    file_path = os.path.join('downloads', model_name + '.gfc')

    with tqdm(total=total_size, unit='iB', unit_scale=True) as t:
        try:
            with open(file_path, 'wb') as f:
                for data in response.iter_content(block_size):
                    t.update(len(data))
                    f.write(data)
            if total_size != 0 and t.n != total_size:
                print("ERROR, something went wrong during the download.")
        except Exception as e:
            print(f"Error during file write: {e}")
            # Optionally, remove partially downloaded file if an error occurs
            if os.path.exists(file_path):
                os.remove(file_path)

    print(f"Downloaded model saved to {file_path}")


100%|██████████| 5.32M/5.32M [00:21<00:00, 246kiB/s]

Downloaded model saved to downloads/GO_CONS_GCF_2_TIM_R6e.gfc


In [7]:
download_ggm()

100%|██████████| 5.32M/5.32M [00:35<00:00, 150kiB/s] 
